# Stage 0: Import Packages

In [1]:
import csv
import random
import json
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from gensim.summarization import keywords
from keybert import KeyBERT
from rake_nltk import Rake
import yake
import spacy
import pke
import textstat
import nltk

/home/dxmonteiro/miniconda3/envs/profextra/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-24 20:47:19.305710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 20:47:19.957592: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-24 20:47:20.957851: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open 

# Stage 1: Extract Data

1.1. Global Values

In [3]:
dataset_scopus = "/home/dxmonteiro/Desktop/WORKSPACE/ProfExtra/input/humanidades_digitais_scopus.csv"

title = 0
citations = 1
doi = 2
link = 3
abstract = 4
keywords = [5,7,9,11,13,15,17,19]
authors = [21,22,23,24,25,26,27,28,29,30,31,32,33,34,35]

list_papers = []
list_authors = []
list_keywords = []

1.2. Auxiliar

In [4]:
def update_list(elementa, list_authors, citations):
  k = 0
  for check in list_authors:
    if check.get('name') == elementa:
      author = {
        'name': elementa,
        'frequence': check.get('frequence') + 1,
        'citations': check.get('citations') + citations
      }
      list_authors[k] = author
      return True
    k += 1
  return False

def get_values(real_authors, list_authors, citations):
  for elementa in real_authors:
    if update_list(elementa, list_authors, citations) == False:
      author = {
        'name': elementa,
        'frequence': 1,
        'citations': citations
      }
      list_authors.append(author)
  return list_authors

1.3. Extraction

In [5]:
with open(dataset_scopus, 'r') as file:
  csvreader = csv.reader(file)
  next(csvreader)
  for row in csvreader:
    if not not row[citations]:
      nplist = np.array(row)
      real_authors = list(filter(None, nplist[authors]))
      real_keywords = list(filter(None, nplist[keywords]))
      new_cit = int(row[citations])
      paper = {
          'doi': nplist[doi],
          'title': nplist[title],
          'abstract': nplist[abstract],
          'link': nplist[link],
          'citations': new_cit,
          'keywords': real_keywords,
          'authors': real_authors
      }
      list_papers.append(paper)
      list_authors = get_values(real_authors, list_authors, new_cit)
      list_keywords = get_values(real_keywords, list_keywords, new_cit)

1.4. Testing Values

In [6]:
print('Number of Papers: ' + str(len(list_papers)))
print('Sample of Papers:')
for i in range(0,10,1):
  print(str(i) + ': ' + str(list_papers[i]))

Number of Papers: 1895
Sample of Papers:
0: {'doi': '10.2979/victorianstudies.64.2.12', 'title': 'Sara Ahmed’s Politics of Citation and Student Scholarship: Uncovering Indigenous Coauthors of British Folklore Collections', 'abstract': 'In 2021, I taught an introductory literary seminar in which students read the folklore of colonized subjects and supernatural fiction by writers of color who had lived under British rule or Western imperial influence. We partnered with Adrian Wisnicki’s digital humanities project One More Voice to create an online anthology and video series that critically frame folklore collections from Japan, India, South Africa, and Jamaica. The students learned about Sara Ahmed’s politics of citation and applied her ideas in giving credit to the Indigenous coauthors who sourced, transcribed, and translated oral folktales for British colonizers. I argue that we should regard the students as scholars who modeled ways of undisciplining Victorian studies in their work on

In [7]:
print('Number of Authors: ' + str(len(list_authors)))
print('Sample of Authors:')
for i in range(0,10,1):
  print(str(i) + ': ' + str(list_authors[i]))

Number of Authors: 3644
Sample of Authors:
0: {'name': 'Ohri I.', 'frequence': 1, 'citations': 0}
1: {'name': 'Bent G.R.', 'frequence': 1, 'citations': 0}
2: {'name': 'Pfaff D.', 'frequence': 1, 'citations': 0}
3: {'name': 'Brooks M.', 'frequence': 2, 'citations': 4}
4: {'name': 'Radpour R.', 'frequence': 1, 'citations': 0}
5: {'name': 'Delaney J.', 'frequence': 1, 'citations': 0}
6: {'name': 'Albertini N.', 'frequence': 1, 'citations': 0}
7: {'name': 'Baldini J.', 'frequence': 1, 'citations': 0}
8: {'name': 'Dal Pino A.', 'frequence': 1, 'citations': 0}
9: {'name': 'Lazzari F.', 'frequence': 1, 'citations': 0}


In [8]:
print('Number of Keywords: ' + str(len(list_keywords)))
print('Sample of Keywords:')
for i in range(0,10,1):
  print(str(i) + ': ' + str(list_keywords[i]))

Number of Keywords: 5146
Sample of Keywords:
0: {'name': 'Digital humanities', 'frequence': 593, 'citations': 2692}
1: {'name': 'Florence', 'frequence': 1, 'citations': 0}
2: {'name': 'LiDAR', 'frequence': 1, 'citations': 0}
3: {'name': 'Model', 'frequence': 1, 'citations': 0}
4: {'name': 'Orsanmichele', 'frequence': 1, 'citations': 0}
5: {'name': 'Photogrammetry', 'frequence': 3, 'citations': 1}
6: {'name': 'Potree', 'frequence': 1, 'citations': 0}
7: {'name': 'Scan', 'frequence': 1, 'citations': 0}
8: {'name': '3D technologies', 'frequence': 1, 'citations': 0}
9: {'name': 'Archaeology', 'frequence': 12, 'citations': 30}


# Stage 2: Process Data

2.1. Get top 10 Papers

In [9]:
list_papers.sort(key=lambda x: x.get('citations'), reverse=True)
top_10_papers = list_papers[:10]
print(top_10_papers)

[{'doi': '10.1177/2053951714528481', 'title': 'Big Data, new epistemologies and paradigm shifts', 'abstract': 'This article examines how the availability of Big Data, coupled with new data analytics, challenges established epistemologies across the sciences, social sciences and humanities, and assesses the extent to which they are engendering paradigm shifts across multiple disciplines. In particular, it critically explores new forms of empiricism that declare ‘the end of theory’, the creation of data-driven rather than knowledge-driven science, and the development of digital humanities and computational social sciences that propose radically different ways to make sense of culture, history, economy and society. It is argued that: (1) Big Data and new data analytics are disruptive innovations which are reconfiguring in many instances how research is conducted; and (2) there is an urgent need for wider critical reflection within the academy on the epistemological implications of the unf

2.2. Get top 10 Authors

In [10]:
list_authors.sort(key=lambda x: x.get('citations'), reverse=True)
top_10_authors = list_authors[:10]
print(top_10_authors)

[{'name': 'Kitchin R.', 'frequence': 1, 'citations': 1062}, {'name': 'Terras M.', 'frequence': 8, 'citations': 218}, {'name': 'Warwick C.', 'frequence': 5, 'citations': 191}, {'name': 'Holmberg K.', 'frequence': 1, 'citations': 182}, {'name': 'Thelwall M.', 'frequence': 1, 'citations': 182}, {'name': 'Anderson C.W.', 'frequence': 1, 'citations': 181}, {'name': 'Welsh A.', 'frequence': 3, 'citations': 154}, {'name': 'Kestemont M.', 'frequence': 5, 'citations': 147}, {'name': 'Hu Y.', 'frequence': 1, 'citations': 144}, {'name': 'Boyd-Graber J.', 'frequence': 1, 'citations': 144}]


2.3. Get top 10 Keywords

In [11]:
list_papers.sort(key=lambda x: x.get('citations'), reverse=True)
top_10_keywords = list_keywords[:10]
print(top_10_keywords)

[{'name': 'Digital humanities', 'frequence': 593, 'citations': 2692}, {'name': 'Florence', 'frequence': 1, 'citations': 0}, {'name': 'LiDAR', 'frequence': 1, 'citations': 0}, {'name': 'Model', 'frequence': 1, 'citations': 0}, {'name': 'Orsanmichele', 'frequence': 1, 'citations': 0}, {'name': 'Photogrammetry', 'frequence': 3, 'citations': 1}, {'name': 'Potree', 'frequence': 1, 'citations': 0}, {'name': 'Scan', 'frequence': 1, 'citations': 0}, {'name': '3D technologies', 'frequence': 1, 'citations': 0}, {'name': 'Archaeology', 'frequence': 12, 'citations': 30}]


2.4. Summaries

In [12]:
class Summarizer():
    def __init__(self, model):
        self.model = model
        self.summarizer = pipeline("summarization", model=self.model)
    
    def get_summary(self, text):
        words = text.split()
        totalwords = len(words)
        summary = self.summarizer(text, max_length = totalwords, do_sample=False)[0].get('summary_text')
        return summary

summ_models = ['facebook/bart-large-cnn', 'sshleifer/distilbart-cnn-12-6', 'philschmid/bart-large-cnn-samsum', 'google/pegasus-large', 'sshleifer/distill-pegasus-cnn-16-4']
list_papers_summ = []

for paper in top_10_papers:
  list_summs = []
  for sm in summ_models:
    ex_summarizer = Summarizer(sm)
    head = {
        'model': sm,
        'summarized_text': ex_summarizer.get_summary(paper.get('abstract'))
    }
    list_summs.append(head)
  new_paper = {
    'doi': paper.get('doi'),
    'title': paper.get('title'),
    'summ_abstract': list_summs,
    'link': paper.get('link'),
    'citations': paper.get('citations'),
    'keywords': paper.get('keywords'),
  }
  list_papers_summ.append(new_paper)

In [13]:
print('Summarized Papers:')
for i in range(0,10,1):
  print(str(i) + ': ' + str(list_papers_summ[i]))

Summarized Papers:
0: {'doi': '10.1177/2053951714528481', 'title': 'Big Data, new epistemologies and paradigm shifts', 'summ_abstract': [{'model': 'facebook/bart-large-cnn', 'summarized_text': 'This article examines how the availability of Big Data, coupled with new data analytics, challenges established epistemologies across the sciences, social sciences and humanities. In particular, it critically explores new forms of empiricism that declare ‘the end of theory’ and the creation of data-driven rather than knowledge-driven science.'}, {'model': 'sshleifer/distilbart-cnn-12-6', 'summarized_text': ' This article examines how the availability of Big Data, coupled with new data analytics, challenges established epistemologies across the sciences, social sciences and humanities . In particular, it critically explores new forms of empiricism that declare ‘the end of theory’ and the creation of data-driven rather than knowledge-driven science .'}, {'model': 'philschmid/bart-large-cnn-samsum'

# Stage 3: Save Current Data

In [16]:
json_dump = {
    "top_10_papers": top_10_papers,
    "top_10_authors": top_10_authors,
    "top_10_keywords": top_10_keywords,
    "summaries": list_papers_summ
}

with open("/home/dxmonteiro/Desktop/WORKSPACE/ProfExtra/output/1stdump.json", "w") as outfile:
    outfile.write(str(json_dump))

# Stage 4: Extracting Keywords with AutoModels

4.1. Reload file data

In [2]:
f = open('/home/dxmonteiro/Desktop/WORKSPACE/ProfExtra/output/1stdump.json')
processed_data = json.load(f)

In [3]:
print(processed_data)

{'top_10_papers': [{'doi': '10.1177/2053951714528481', 'title': 'Big Data, new epistemologies and paradigm shifts', 'abstract': 'This article examines how the availability of Big Data, coupled with new data analytics, challenges established epistemologies across the sciences, social sciences and humanities, and assesses the extent to which they are engendering paradigm shifts across multiple disciplines. In particular, it critically explores new forms of empiricism that declare ‘the end of theory’, the creation of data-driven rather than knowledge-driven science, and the development of digital humanities and computational social sciences that propose radically different ways to make sense of culture, history, economy and society. It is argued that: (1) Big Data and new data analytics are disruptive innovations which are reconfiguring in many instances how research is conducted; and (2) there is an urgent need for wider critical reflection within the academy on the epistemological impli

4.2. Get top10_papers & top10_summaries

In [4]:
processed_10papers = processed_data.get('top_10_papers')
processed_10summaries = processed_data.get('summaries')

In [5]:
print(processed_10papers)

[{'doi': '10.1177/2053951714528481', 'title': 'Big Data, new epistemologies and paradigm shifts', 'abstract': 'This article examines how the availability of Big Data, coupled with new data analytics, challenges established epistemologies across the sciences, social sciences and humanities, and assesses the extent to which they are engendering paradigm shifts across multiple disciplines. In particular, it critically explores new forms of empiricism that declare ‘the end of theory’, the creation of data-driven rather than knowledge-driven science, and the development of digital humanities and computational social sciences that propose radically different ways to make sense of culture, history, economy and society. It is argued that: (1) Big Data and new data analytics are disruptive innovations which are reconfiguring in many instances how research is conducted; and (2) there is an urgent need for wider critical reflection within the academy on the epistemological implications of the unf

In [7]:
print(processed_10summaries)

[{'doi': '10.1177/2053951714528481', 'title': 'Big Data, new epistemologies and paradigm shifts', 'summ_abstract': [{'model': 'facebook/bart-large-cnn', 'summarized_text': 'This article examines how the availability of Big Data, coupled with new data analytics, challenges established epistemologies across the sciences, social sciences and humanities. In particular, it critically explores new forms of empiricism that declare ‘the end of theory’ and the creation of data-driven rather than knowledge-driven science.'}, {'model': 'sshleifer/distilbart-cnn-12-6', 'summarized_text': ' This article examines how the availability of Big Data, coupled with new data analytics, challenges established epistemologies across the sciences, social sciences and humanities . In particular, it critically explores new forms of empiricism that declare ‘the end of theory’ and the creation of data-driven rather than knowledge-driven science .'}, {'model': 'philschmid/bart-large-cnn-samsum', 'summarized_text': 

In [9]:
# 1. RAKE
def rake_extractor(text):
    """
    Uses Rake to extract the top 5 keywords from a text
    Arguments: text (str)
    Returns: list of keywords (list)
    """
    r = Rake()
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()[:10]

# 2. YAKE
def yake_extractor(text):
    """
    Uses YAKE to extract the top 5 keywords from a text
    Arguments: text (str)
    Returns: list of keywords (list)
    """
    keywords = yake.KeywordExtractor(lan="en", n=3, windowsSize=3, top=10).extract_keywords(text)
    results = []
    for scored_keywords in keywords:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword) 
    return results 


# 3. PositionRank
def position_rank_extractor(text):
    """
    Uses PositionRank to extract the top 5 keywords from a text
    Arguments: text (str)
    Returns: list of keywords (list)
    """
    # define the valid Part-of-Speeches to occur in the graph
    pos = {'NOUN', 'PROPN', 'ADJ', 'ADV'}
    extractor = pke.unsupervised.PositionRank()
    extractor.load_document(text, language='en')
    extractor.candidate_selection(maximum_word_number=5)
    # 4. weight the candidates using the sum of their word's scores that are
    #    computed using random walk biaised with the position of the words
    #    in the document. In the graph, nodes are words (nouns and
    #    adjectives only) that are connected if they occur in a window of
    #    3 words.
    extractor.candidate_weighting(window=3, pos=pos)
    # 5. get the 5-highest scored candidates as keyphrases
    keyphrases = extractor.get_n_best(n=10)
    results = []
    for scored_keywords in keyphrases:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword) 
    return results 

# 4. SingleRank
def single_rank_extractor(text):
    """
    Uses SingleRank to extract the top 5 keywords from a text
    Arguments: text (str)
    Returns: list of keywords (list)
    """
    pos = {'NOUN', 'PROPN', 'ADJ', 'ADV'}
    extractor = pke.unsupervised.SingleRank()
    extractor.load_document(text, language='en')
    extractor.candidate_selection(pos=pos)
    extractor.candidate_weighting(window=3, pos=pos)
    keyphrases = extractor.get_n_best(n=10)
    results = []
    for scored_keywords in keyphrases:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword) 
    return results 

# 5. MultipartiteRank
def multipartite_rank_extractor(text):
    """
    Uses MultipartiteRank to extract the top 5 keywords from a text
    Arguments: text (str)
    Returns: list of keywords (list)
    """
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(text, language='en')
    pos = {'NOUN', 'PROPN', 'ADJ', 'ADV'}
    extractor.candidate_selection(pos=pos)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1, threshold=0.74, method='average')
    keyphrases = extractor.get_n_best(n=10)
    results = []
    for scored_keywords in keyphrases:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword) 
    return results

# 6. TopicRank
def topic_rank_extractor(text):
    """
    Uses TopicRank to extract the top 5 keywords from a text
    Arguments: text (str)
    Returns: list of keywords (list)
    """
    extractor = pke.unsupervised.TopicRank()
    extractor.load_document(text, language='en')
    pos = {'NOUN', 'PROPN', 'ADJ', 'ADV'}
    extractor.candidate_selection(pos=pos)
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=10)
    results = []
    for scored_keywords in keyphrases:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword) 
    return results

# 7. KeyBERT
def keybert_extractor(text):
    bert = KeyBERT()
    """
    Uses KeyBERT to extract the top 5 keywords from a text
    Arguments: text (str)
    Returns: list of keywords (list)
    """
    keywords = bert.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words="english", top_n=10)
    results = []
    for scored_keywords in keywords:
        for keyword in scored_keywords:
            if isinstance(keyword, str):
                results.append(keyword)
    return results

In [10]:
class KeywordExtractor():
    def __init__(self, model):
        self.model = model
        
    def get_keywords(self, text):
        if self.model == 'KEYBERT':
            head = {'KEYBERT': keybert_extractor(text)}
        elif self.model == 'TOPIC':
            head = {'TOPIC RANK': topic_rank_extractor(text)} 
        elif self.model == 'MULTIPARTITE':
            head = {'MULTIPARTITE RANK': multipartite_rank_extractor(text)}
        elif self.model == 'SINGLE':
            head = {'SINGLE RANK': single_rank_extractor(text)}
        elif self.model == 'YAKE':
            head = {'YAKE': yake_extractor(text)}
        elif self.model == 'RAKE':
            head = {'RAKE': rake_extractor(text)}
        elif self.model == 'POSITION':
            head = {'POSITION RANK': position_rank_extractor(text)}
        else:
            head = {}
            
        return head

In [11]:
def final_process_data(json_data):
    
    keyword_models=['KEYBERT', 'YAKE', 'RAKE', 'POSITION', 'SINGLE', 'MULTIPARTITE', 'TOPIC']
    
    data = []
    
    for i in json_data:
        ex_keyword = KeywordExtractor(keyword)
        s_abstract = ex_summarizer.get_summary(i.get('original').get('abstract'))
        s_full = get_fulltext_summary(i, ex_summarizer)
        s_article = ex_summarizer.get_summary(i.get('article').get('text'))
        head = {
            "abstract": {
                "title": i.get('original').get('title'),
                "url": i.get('original').get('url'),
                "abstract": i.get('original').get('abstract'),
                "tokens": len(nltk.word_tokenize(i.get('original').get('abstract'))),
            },
            "full_text": {
                "title": i.get('original').get('title'),
                "url": i.get('original').get('url'),
                "keywords": i.get('original').get('keywords'),
                "text": i.get('original').get('text'),
                "sections": i.get('original').get('sections'),
                "tokens": len(nltk.word_tokenize(i.get('original').get('text'))),
            },
            "article": {
                "title": i.get('article').get('title'),
                "url": i.get('article').get('url'),
                "keywords": i.get('article').get('keywords'),
                "text": i.get('article').get('text'),
                "tokens": len(nltk.word_tokenize(i.get('article').get('text'))),
            },
            "summaries": {
                "abstract": {
                    "text": s_abstract,
                    "tokens": len(nltk.word_tokenize(s_abstract)),
                },
                "full_text": {
                    "text": s_full,
                    "tokens": len(nltk.word_tokenize(s_full)),
                },
                "article": {
                    "text": s_article,
                    "tokens": len(nltk.word_tokenize(s_article)),
                }
            },
            "keywords": {
                "abstract": ex_keyword.get_keywords(i.get('original').get('abstract')),
                "full_text": ex_keyword.get_keywords(i.get('original').get('text')),
                "article": ex_keyword.get_keywords(i.get('article').get('text'))
            }
        }
        data.append(head)
    return data